In [1]:
import MutagenesisForge as mf

TypeError: context() missing 12 required positional arguments: 'fasta', 'vcf', 'bed', 'model', 'alpha', 'beta', 'gamma', 'pi_a', 'pi_c', 'pi_g', 'pi_t', and 'context_model'